# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [2]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['Left', 'HeadBack', 'Belly', 'Stand', 'Frog', 'StandInit', 'Back', 'Right', 'Knee', 'Sit', 'Crouch']


In [3]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    return data, target

In [4]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE

for i in xrange(len(classes)):
    (data, target) = load_pose_data(i)
    all_data += data
    all_target += target

print 'total number of data', len(all_data)
print all_target

total number of data 222
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


In [5]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [6]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [23]:

training_set = []
target_set = []
'''
for i in xrange(n_training_data):
    index = training_data[i]
    training_set.append(all_data[index])
    target_set.append(all_target[index])
'''

permutations = []
swapped = 0
for i in xrange(len(all_target)):
    if(all_target[swapped] != all_target[i]):
        tmp = np.random.permutation(i - swapped)
        tmp += swapped
        n_train = int(0.8 * len(tmp))
        permutations += tmp[:n_train].tolist()
        swapped = i
    if(len(all_target) == i + 1):
        tmp = np.random.permutation(i - swapped + 1)
        tmp += swapped
        n_train = int(0.8 * len(tmp))
        permutations += tmp[:n_train].tolist()
        swapped = i
        
for (i, p) in enumerate(permutations):
    training_set.append(all_data[p])
    target_set.append(all_target[p])
print permutations
clf.fit(training_set, target_set)

[0, 1, 4, 7, 8, 15, 2, 6, 16, 19, 11, 5, 13, 10, 18, 12, 22, 23, 21, 24, 29, 20, 28, 27, 46, 37, 34, 41, 45, 33, 36, 35, 44, 30, 32, 47, 31, 39, 48, 51, 55, 54, 57, 59, 50, 58, 52, 67, 60, 61, 69, 66, 64, 68, 62, 87, 84, 82, 111, 72, 120, 97, 108, 103, 107, 91, 98, 80, 79, 92, 116, 119, 114, 88, 90, 96, 106, 85, 74, 104, 83, 76, 112, 89, 93, 121, 99, 81, 95, 101, 78, 110, 113, 109, 75, 94, 143, 140, 134, 124, 139, 144, 137, 136, 122, 128, 132, 142, 133, 138, 129, 131, 127, 141, 155, 145, 151, 146, 147, 153, 148, 150, 163, 157, 158, 156, 164, 161, 159, 162, 191, 174, 170, 186, 167, 190, 185, 188, 182, 175, 172, 177, 184, 169, 187, 173, 189, 178, 168, 180, 199, 202, 192, 211, 210, 215, 205, 217, 219, 200, 195, 194, 207, 216, 213, 206, 201, 221, 218, 212, 204, 197, 193, 196]


SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### predicting

In [24]:
clf.predict(all_data[-1]), all_target[-1]

/home/alex/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(array([10]), 10)

In [25]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [26]:
expected = []
predicted = []
# YOUR CODE HERE
test_data = []
'''
for i in xrange(n_training_data):
    test_index = training_data[i]
    expected.append(all_target[test_index])
    #tmp = np.array(all_data[test_index]).reshape(1, -1)
    test_data.append(all_data[test_index])

'''

for (i, p) in enumerate(permutations):
    expected.append(all_target[p])
    test_data.append(all_data[p])

predicted = clf.predict(test_data)
evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        16
          1       1.00      1.00      1.00         8
          2       1.00      1.00      1.00        15
          3       1.00      1.00      1.00         8
          4       1.00      1.00      1.00         8
          5       1.00      1.00      1.00        41
          6       1.00      1.00      1.00        18
          7       1.00      0.88      0.93         8
          8       1.00      1.00      1.00         8
          9       1.00      1.00      1.00        20
         10       0.96      1.00      0.98        24

avg / total       0.99      0.99      0.99       174


Confusion matrix:
[[16  0  0  0  0  0  0  0  0  0  0]
 [ 0  8  0  0  0  0  0  0  0  0  0]
 [ 0  0 15  0  0  0  0  0  0  0  0]
 [ 0  0  0  8  0  0  0  0  0  0  0]
 [ 0  0  0  0  8  0  0  0  0  0  0]
 [ 0  0  0  0  0 41  0  0  0  0  0]
 [ 0  0  0  0  0  0 18  0  0  0  0]
 [ 0  0  0  0 

## 4. Evaluate on the test data

In [28]:
expected = []
predicted = []
# YOUR CODE HERE
test_data = []
'''for i in xrange(len(all_data)):
    if(not(i in training_data)):
        expected.append(all_target[i])
        #tmp = np.array(all_data[i]).reshape(1, -1)
        test_data.append(all_data[i])'''

print permutations
for i in xrange(len(all_data)):
    if(i not in permutations):
        expected.append(all_target[i])
        test_data.append(all_data[i])
        
print expected       
predicted = clf.predict(test_data)

evaluate(expected, predicted)

[0, 1, 4, 7, 8, 15, 2, 6, 16, 19, 11, 5, 13, 10, 18, 12, 22, 23, 21, 24, 29, 20, 28, 27, 46, 37, 34, 41, 45, 33, 36, 35, 44, 30, 32, 47, 31, 39, 48, 51, 55, 54, 57, 59, 50, 58, 52, 67, 60, 61, 69, 66, 64, 68, 62, 87, 84, 82, 111, 72, 120, 97, 108, 103, 107, 91, 98, 80, 79, 92, 116, 119, 114, 88, 90, 96, 106, 85, 74, 104, 83, 76, 112, 89, 93, 121, 99, 81, 95, 101, 78, 110, 113, 109, 75, 94, 143, 140, 134, 124, 139, 144, 137, 136, 122, 128, 132, 142, 133, 138, 129, 131, 127, 141, 155, 145, 151, 146, 147, 153, 148, 150, 163, 157, 158, 156, 164, 161, 159, 162, 191, 174, 170, 186, 167, 190, 185, 188, 182, 175, 172, 177, 184, 169, 187, 173, 189, 178, 168, 180, 199, 202, 192, 211, 210, 215, 205, 217, 219, 200, 195, 194, 207, 216, 213, 206, 201, 221, 218, 212, 204, 197, 193, 196]
[0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 8, 8, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10]
Classification report:
             precision    recall  f1-score 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [29]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [30]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1]), all_target[-1]

/home/alex/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(array([10]), 10)